In [7]:
INSTALL_PACKAGES = False
if INSTALL_PACKAGES:
    ! pip install --user rightmove-webscraper --quiet
    ! pip install pytesseract --quiet
    ! pip install requests --quiet
    ! pip install bs4 --quiet
    ! pip install pywebcopy --quiet
    ! pip install pycurl --quiet
    ! pip install selenium
    ! pip install keyboard
    ! pip install nltk

# Imports

In [91]:
from PIL import ImageFilter

In [1]:
import time
from selenium import webdriver
import keyboard

In [78]:
import pytesseract

tesseract_install_path = r"C:\Program Files\Tesseract-OCR\tesseract.exe" # needs updating depending on x64-x86 and OS
pytesseract.pytesseract.tesseract_cmd = tesseract_install_path

In [ ]:
import nltk

In [3]:
from rightmove_webscraper import RightmoveData

In [4]:
import os
import glob
import shutil
import re
import requests
from bs4 import BeautifulSoup
import ukpropsearch_utils as utils
import importlib
importlib.reload(utils)

<module 'ukpropsearch_utils' from 'C:\\Users\\alexl\\Documents\\GitHub\\UK-property-searcher\\ukpropsearch_utils.py'>

## Rightmove string building

In [5]:
london_r3miles_url = "https://www.rightmove.co.uk/property-for-sale/find.html?minBedrooms=2&keywords=&dontShow=sharedOwnership%2Cretirement&channel=BUY&index=0&retirement=false&maxBedrooms=2&includeSSTC=false&partBuyPartRent=false&sortType=2&minPrice=400000&viewType=LIST&maxPrice=450000&radius=3.0&maxDaysSinceAdded=14&locationIdentifier=REGION%5E87490"

In [6]:
london_r3miles = RightmoveData(london_r3miles_url)
london_r3miles_results = london_r3miles.get_results
london_r3miles.summary()

,number_bedrooms,count,mean
0,2,575,448834.695652


In [150]:
test_property = london_r3miles_results.iloc[3]
test_property_url = test_property.url
test_property_url = test_property_url.split('#')[0] + "#/"
test_property_floorplan_url = test_property_url + "floorplan"
test_property_hashkey = re.sub(r'[^\w]', '', f"{test_property.address}")
print(test_property_hashkey)
print(test_property_url)
print(test_property_floorplan_url)

RaleighRoadLondonN8
http://www.rightmove.co.uk/properties/121008602#/
http://www.rightmove.co.uk/properties/121008602#/floorplan


In [213]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

listUrls = [test_property_floorplan_url]
browser = webdriver.Chrome()
urls=[]

for url in listUrls:
    browser.get(url)
    soup = BeautifulSoup(browser.page_source,"html.parser")
    for link in soup.find_all('img'):
        urls.append(link['src'])

In [214]:
urls

['https://media.rightmove.co.uk/37k/36308/121008602/36308_HAR220115_IMG_00_0000.jpeg',
 'https://media.rightmove.co.uk/37k/36308/121008602/36308_HAR220115_IMG_01_0000.jpeg',
 'https://media.rightmove.co.uk/37k/36308/121008602/36308_HAR220115_IMG_02_0000.jpeg',
 'https://media.rightmove.co.uk/37k/36308/121008602/36308_HAR220115_IMG_03_0000.jpeg',
 'https://media.rightmove.co.uk/37k/36308/121008602/36308_HAR220115_IMG_04_0000.jpeg',
 'https://media.rightmove.co.uk/37k/36308/121008602/36308_HAR220115_IMG_05_0000.jpeg',
 'https://media.rightmove.co.uk/37k/36308/121008602/36308_HAR220115_IMG_06_0000.jpeg',
 'https://media.rightmove.co.uk/37k/36308/121008602/36308_HAR220115_IMG_07_0000.jpeg',
 'https://media.rightmove.co.uk/37k/36308/121008602/36308_HAR220115_IMG_08_0000.jpeg',
 'https://media.rightmove.co.uk/37k/36308/121008602/36308_HAR220115_IMG_09_0000.jpeg',
 'https://media.rightmove.co.uk/37k/36308/121008602/36308_HAR220115_IMG_10_0000.jpeg',
 'https://media.rightmove.co.uk/37k/36308/1

In [216]:
test_property_floorplan_url = [url for url in urls if "FLP" in url][0]
test_property_floorplan_url

'https://media.rightmove.co.uk/37k/36308/121008602/36308_HAR220115_FLP_00_0001.jpeg'

In [151]:
cache_folder = f'{os.getcwd()}\\cache'
images_folder = f'{os.getcwd()}\\images'
floorplan_imgs_hd_download_folder = f"{cache_folder}\\{test_property_hashkey}"
floorplan_imgs_hd_download_fullpath = f"{floorplan_imgs_hd_download_folder}\\{test_property_hashkey}.html"

# Make sure cache folder exists
if not os.path.exists(cache_folder):
    os.mkdir(cache_folder)

# Make sure images folder exists
if not os.path.exists(images_folder):
    os.mkdir(images_folder)

# Floorplan destination image fullpath
floorplan_destination_img_filename = test_property_hashkey
floorplan_destination_img_path = os.path.join(images_folder, floorplan_destination_img_filename)

In [152]:
floorplan_destination_img_path

'C:\\Users\\alexl\\Documents\\GitHub\\UK-property-searcher\\images\\RaleighRoadLondonN8'

In [153]:
importlib.reload(utils)
def wait_for_floorplan_download_finish():
    floorplan_img_hd_path = ""
    while not os.path.exists(floorplan_img_hd_path):
        try:
            image_paths = utils.find_files(['*.jpeg', "*.png", "*.jpg"], floorplan_imgs_hd_download_folder)
            floorplan_img_hd_path = [image_path for image_path in image_paths if "FLP" in image_path][0]
            time.sleep(0.1)
        except:
            pass

    return floorplan_img_hd_path

In [154]:
floorplan_destination_img_path

'C:\\Users\\alexl\\Documents\\GitHub\\UK-property-searcher\\images\\RaleighRoadLondonN8'

In [155]:
if not utils.file_exists_with_or_without_extension(floorplan_destination_img_path):
    if not os.path.exists(floorplan_imgs_hd_download_fullpath):
        if not os.path.exists(floorplan_imgs_hd_download_folder):
            os.makedirs(floorplan_imgs_hd_download_folder)

        # Open Chrome at url and maximise
        driver = webdriver.Chrome()  # Optional argument, if not specified will search path.
        driver.get(test_property_floorplan_url);
        driver.maximize_window()

        # Save the whole page
        time.sleep(0.1)
        keyboard.press_and_release('ctrl+s')
        time.sleep(0.4)
        keyboard.write(floorplan_imgs_hd_download_fullpath)
        keyboard.press_and_release('enter')
        time.sleep(5)
        floorplan_img_hd_path = wait_for_floorplan_download_finish()
        driver.close()

    # Copy img to destination path
    floorplan_destination_img_path = floorplan_destination_img_path + "." + floorplan_img_hd_path.split(".")[-1] # add extension from downloaded file
    shutil.copy2(floorplan_img_hd_path, floorplan_destination_img_path)

    # Clean the cache from this download
    shutil.rmtree(floorplan_imgs_hd_download_folder)

In [156]:
floorplan_destination_img_path

'C:\\Users\\alexl\\Documents\\GitHub\\UK-property-searcher\\images\\RaleighRoadLondonN8.jpeg'

# Text recognition

In [157]:
upscaled_img_path = f"{floorplan_destination_img_path.split('.')[0]}_upscaled.{floorplan_destination_img_path.split('.')[-1]}"
upscaled_img_path

'C:\\Users\\alexl\\Documents\\GitHub\\UK-property-searcher\\images\\RaleighRoadLondonN8_upscaled.jpeg'

In [158]:
from PIL import Image
c = Image.open(floorplan_destination_img_path)
new_size = (int(c.width*3), int(c.height*3))
new_size

(1464, 3051)

In [159]:
# Upscale image
c = c.filter(ImageFilter.SHARPEN)
d = c.resize(new_size, resample=Image.ADAPTIVE)

In [160]:
d = d.filter(ImageFilter.SHARPEN)

In [161]:
d.save(upscaled_img_path, format='JPEG', subsampling=0, quality=100)

In [162]:
# Simple image to string
floorplan_text = pytesseract.image_to_string(Image.open(upscaled_img_path)).split("\n")

In [163]:
floorplan_text = [t.replace("\n", "") for t in floorplan_text if t]
floorplan_text

['Raleigh Road, N8',
 'Approx. Gross Internal Floor Area 695 sq. ft / 64.60 sq. m',
 'BEDROOM 2',
 '111 x 10°1',
 '(3.37m x 3.07m)',
 'KITCHEN',
 "~ rey 7'9x7'2",
 '_ (2.34m x 2.18m)',
 'BATHROOM',
 '2x7',
 'MASTER',
 'BEDROOM',
 '42°41 x 10°11',
 'fl (3.92m x 3.28m)',
 'RECEPTION ROOM',
 '461 x 144',
 '(4.88m x 4.28m)',
 '« »',
 'GROUND FLOOR',
 'GROSS INTERNAL FIRST FLOOR',
 'FLOOR AREA 15 SQ FT GROSS INTERNAL',
 'FLOOR AREA 680 SQ FT',
 'All Ue anil of bn bats, Aleit fabings an spelancts, ;',
 'including their size and location, are shovm as standard sizes and do no',
 'constitute any warranty or representation oy the seller, their agent or Wi',
 'CP Creative. An inondiy purchaser aust satisty himself inspection or Tal acorn)',
 'otherwise a$ to the correctness of the infarmation contained in these plans.',
 'This plan is for ittustrative purposes only and should be used as such by',
 'any prospective purchasers.']

In [164]:
from nltk.corpus import wordnet
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alexl\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [165]:
room_hyponyms = wordnet.synsets('room')[0].hyponyms()

In [166]:
room_hyponyms = [hyp.lemma_names() for hyp in room_hyponyms]

In [167]:
# selectMany
room_names_set = set([hpy.replace('_', ' ') for hyp_lemma in room_hyponyms for hpy in hyp_lemma])

In [168]:
# Utility spaces hyponyms
utility_space_hyponyms = ["driveway", "garage", "utility room", "utility"]

In [169]:
# Outdoor space hyponyms
outdoor_space_hyponyms = ["patio", "garden", "terrace", "balcony"]
outdoor_space_hyponyms_communal = [f"communal {hyp}" for hyp in outdoor_space_hyponyms]
outdoor_space_hyponyms_private = [f"private {hyp}" for hyp in outdoor_space_hyponyms]

In [170]:
# Add more names
room_names_set.update(["studio", "garden studio"])
room_names_set.update(outdoor_space_hyponyms)
room_names = list(room_names_set)

In [171]:
floorplan_text

['Raleigh Road, N8',
 'Approx. Gross Internal Floor Area 695 sq. ft / 64.60 sq. m',
 'BEDROOM 2',
 '111 x 10°1',
 '(3.37m x 3.07m)',
 'KITCHEN',
 "~ rey 7'9x7'2",
 '_ (2.34m x 2.18m)',
 'BATHROOM',
 '2x7',
 'MASTER',
 'BEDROOM',
 '42°41 x 10°11',
 'fl (3.92m x 3.28m)',
 'RECEPTION ROOM',
 '461 x 144',
 '(4.88m x 4.28m)',
 '« »',
 'GROUND FLOOR',
 'GROSS INTERNAL FIRST FLOOR',
 'FLOOR AREA 15 SQ FT GROSS INTERNAL',
 'FLOOR AREA 680 SQ FT',
 'All Ue anil of bn bats, Aleit fabings an spelancts, ;',
 'including their size and location, are shovm as standard sizes and do no',
 'constitute any warranty or representation oy the seller, their agent or Wi',
 'CP Creative. An inondiy purchaser aust satisty himself inspection or Tal acorn)',
 'otherwise a$ to the correctness of the infarmation contained in these plans.',
 'This plan is for ittustrative purposes only and should be used as such by',
 'any prospective purchasers.']

In [172]:
"bedroom" in floorplan_text[13].lower()

False

In [173]:
floorplan_text

['Raleigh Road, N8',
 'Approx. Gross Internal Floor Area 695 sq. ft / 64.60 sq. m',
 'BEDROOM 2',
 '111 x 10°1',
 '(3.37m x 3.07m)',
 'KITCHEN',
 "~ rey 7'9x7'2",
 '_ (2.34m x 2.18m)',
 'BATHROOM',
 '2x7',
 'MASTER',
 'BEDROOM',
 '42°41 x 10°11',
 'fl (3.92m x 3.28m)',
 'RECEPTION ROOM',
 '461 x 144',
 '(4.88m x 4.28m)',
 '« »',
 'GROUND FLOOR',
 'GROSS INTERNAL FIRST FLOOR',
 'FLOOR AREA 15 SQ FT GROSS INTERNAL',
 'FLOOR AREA 680 SQ FT',
 'All Ue anil of bn bats, Aleit fabings an spelancts, ;',
 'including their size and location, are shovm as standard sizes and do no',
 'constitute any warranty or representation oy the seller, their agent or Wi',
 'CP Creative. An inondiy purchaser aust satisty himself inspection or Tal acorn)',
 'otherwise a$ to the correctness of the infarmation contained in these plans.',
 'This plan is for ittustrative purposes only and should be used as such by',
 'any prospective purchasers.']

In [174]:
room_names = [name for name in room_names if "floor" not in name]

In [175]:
# find room names in the floorplan
room_indexes = sorted(list(set([floorplan_text.index(t) for room_name in room_names for t in floorplan_text if len(t) < 30 and room_name in t.lower()])))
room_indexes

[2, 5, 8, 11, 14]

In [176]:
rooms_found = [floorplan_text[index] for index in room_indexes]
rooms_found

['BEDROOM 2', 'KITCHEN', 'BATHROOM', 'BEDROOM', 'RECEPTION ROOM']

In [177]:
# Floor number
# Generate all ordinal names for floor
floor_ordinal_names = ["basement", "lower ground", "ground"]
floor_ordinal_names.extend(["roof"])
floor_ordinal_names.extend(utils.ordinal(i) for i in range(1,100))

In [192]:
internal_area_names = ["approx. gross internal floor area", "approx. gross internal floor area", "internal floor area", "floor area"]
internal_area_names_text = list(set([text for text in floorplan_text for ian in internal_area_names if ian in text.lower()]))
internal_area_names_text.sort(key=lambda s: len(s))
internal_area_names_text.reverse()
internal_area_names_text = internal_area_names_text[0]
internal_area_names_text

'Approx. Gross Internal Floor Area 695 sq. ft / 64.60 sq. m'

In [197]:
area_unit_names = ["sqm", "sqmt", "sqft", "sqf", "sq. ft", "sq. m"]

In [198]:
internal_area_names_text

'Approx. Gross Internal Floor Area 695 sq. ft / 64.60 sq. m'

In [200]:
regex = re.compile(rf'[0-9\.\s]+(?:{"|".join(area_unit_names)})')
area_texts = regex.findall(internal_area_names_text.lower())
area_texts = [at.replace(" ","") for at in area_texts]
area_texts

['695sq.ft', '64.60sq.m']

In [204]:
areas = [re.findall(r'[A-Za-z]+|\d+(?:\.\d+)?', s) for s in area_texts]
areas = ["".join(a) for a in areas]
areas = [re.findall(r'[A-Za-z]+|\d+(?:\.\d+)?', s) for s in areas]
areas

[['695', 'sqft'], ['64.60', 'sqm']]